In [1]:
!pip install sentence-transformers datasets

     |████████████████████████████████| 92kB 5.5MB/s 
     |████████████████████████████████| 245kB 30.2MB/s 
     |████████████████████████████████| 2.5MB 40.0MB/s 
     |████████████████████████████████| 1.2MB 50.2MB/s 
     |████████████████████████████████| 245kB 54.1MB/s 
     |████████████████████████████████| 122kB 55.2MB/s 
     |████████████████████████████████| 901kB 48.8MB/s 
     |████████████████████████████████| 3.3MB 42.8MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-cp37-none-any.whl size=126711 sha256=a533d92fe07ef629089e0f3d0fcfcd91fbf65420c00b7c46a351fcc4c3259a46
  Stored in directory: /root/.cache/pip/wheels/38/d2/98/d191289a877a34c68aa67e05179521e060f96394a3e9336be6
Successfully built sentence-transformers
ERROR: transformers 4.8.2 has requirement huggingface-hub==0.0.12, but you'll have huggingface-hub 0.0.13 which is incompatible.


In [2]:
from datasets import load_dataset 
imdb = load_dataset("imdb")

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a. Subsequent calls will reuse this data.


In [3]:
imdb = imdb.shuffle()

In [4]:
imdb_x = [x for x in imdb['train'][:1000]['text']] 
labels = [x for x in imdb['train'][:1000]['label']] 
import pandas as pd 
pd.DataFrame(imdb_x, 
             columns=["text"]).to_excel( 
                                 "imdb.xlsx", 
                                  index=None) 

In [5]:
pd.read_excel("KHMER.xlsx") 

,text
0,ខណៈពេលដែលជំនាន់ដើម ១៩៣២ ជាមួយប្រេសស្តុនហ្វតគឺល...
1,ភាពយន្តអាមេរិចដែលនិយាយច្រើនបំផុតនៅក្នុងមហោស្រព...
2,ប៊ីងហ្គោគឺជាហ្គេមល្បែងអុកគឺជាឈ្មោះ។ កម្រមានអេក...
3,មានហេតុផលជាច្រើនដែលធ្វើឱ្យខ្ញុំវាយតម្លៃសោណុសយ៉...
4,ខ្ញុំគិតថាការមើលវីដេអូការងារស្តីពីការអនុលោមតាម...
...,...
995,ខ្ញុំបានឃើញខ្សែភាពយន្តនេះជាលើកដំបូងនៅទូរទស្សន៍...
996,ខ្សែភាពយន្តនេះល្អណាស់។ ខ្ញុំមិនមែនជាអ្នកគាំទ្រ...
997,ផលប៉ះពាល់ភ្លាមៗគឺជាខ្សែភាពយន្តទី ៤ នៃខ្សែភាពយន...
998,កាលពី ៣ ឆ្នាំមុនរ៉ាជែល (ថេរ៉េសហ្វ្រែលវែល) កំពុ...


In [6]:
imdb_khmer = list(pd.read_excel("KHMER.xlsx").text) 

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
train_x, test_x, train_y, test_y, khmer_train, khmer_test = train_test_split(imdb_x, labels, imdb_khmer, test_size = 0.2, random_state = 1) 

In [9]:
from sentence_transformers import SentenceTransformer 
model = SentenceTransformer("stsb-xlm-r-multilingual") 

In [10]:
encoded_train = model.encode(train_x) 
encoded_test = model.encode(test_x) 
encoded_khmer_test = model.encode(khmer_test) 

In [11]:
import numpy as np 
train_y = np.array(train_y) 
test_y = np.array(test_y) 

In [12]:
import tensorflow as tf 
input_ = tf.keras.layers.Input((768,)) 
classification = tf.keras.layers.Dense( 
                                1, 
                                activation="sigmoid")(input_) 
classification_model = tf.keras.Model(input_, classification) 
classification_model.compile( 
               loss=tf.keras.losses.BinaryCrossentropy(), 
               optimizer="Adam", 
               metrics=["accuracy", "Precision", "Recall"]) 

In [13]:
classification_model.fit( 
                     x = encoded_train, 
                     y = train_y, 
                     validation_data=(encoded_test, test_y), 
                     epochs = 20) 

Epoch 1/20
25/25 [==============================] - 4s 15ms/step - loss: 0.6512 - accuracy: 0.6025 - precision: 0.6229 - recall: 0.6110 - val_loss: 0.5918 - val_accuracy: 0.7100 - val_precision: 0.7257 - val_recall: 0.7523
Epoch 2/20
25/25 [==============================] - 0s 4ms/step - loss: 0.5478 - accuracy: 0.7362 - precision: 0.7321 - recall: 0.7828 - val_loss: 0.5429 - val_accuracy: 0.7100 - val_precision: 0.7684 - val_recall: 0.6697
Epoch 3/20
25/25 [==============================] - 0s 4ms/step - loss: 0.5097 - accuracy: 0.7638 - precision: 0.7738 - recall: 0.7757 - val_loss: 0.5219 - val_accuracy: 0.7150 - val_precision: 0.7653 - val_recall: 0.6881
Epoch 4/20
25/25 [==============================] - 0s 4ms/step - loss: 0.4894 - accuracy: 0.7912 - precision: 0.7986 - recall: 0.8043 - val_loss: 0.5150 - val_accuracy: 0.7150 - val_precision: 0.7653 - val_recall: 0.6881
Epoch 5/20
25/25 [==============================] - 0s 4ms/step - loss: 0.4749 - accuracy: 0.7912 - precision: 

In [14]:
classification_model.evaluate(x = encoded_khmer_test,
                              y = test_y) 

7/7 [==============================] - 0s 3ms/step - loss: 0.5945 - accuracy: 0.7250 - precision: 0.7015 - recall: 0.8624


[0.5944913625717163,
 0.7250000238418579,
 0.7014925479888916,
 0.8623853325843811]